In [ ]:
import os, sys
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import shutil
import collections
from pathlib import Path

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Loading packages

In [ ]:
import sys
from pathlib import Path

here_path = Path().resolve()
repo_path = here_path.parents[1]
sys.path.append(str(repo_path))

In [ ]:
from py.utils import verifyDir, verifyFile, verifyType

In [ ]:
from py.config import Config

cfg = Config()

np.random.seed(cfg.RANDOM_STATE)
cfg.DATA_PATH, cfg.MODEL_PATH

In [ ]:
QSCORE_PATH=f"{cfg.DATA_PATH}pp1/Qscores/"
IMAGES_PATH = f"{cfg.DATA_PATH}pp1/images/"
FEATURES_PATH = f"{cfg.MODEL_PATH}pp1/{cfg.YEAR_STUDIED}/features/"
MODEL_PATH = f"{cfg.MODEL_PATH}pp1/{cfg.YEAR_STUDIED}/linear/"

In [ ]:
verifyDir(MODEL_PATH)

### Loading data

In [ ]:
%%time
import pickle

with open(f"{FEATURES_PATH}features.pkl", "rb") as f:
    features_dict = pickle.load(f)
del features_dict["sift"]

In [ ]:
features_df = pd.DataFrame(data=features_dict)
features_df["image_id"] = features_df["image_id"].apply(str)
features_df

In [ ]:
%%time
data_df = pd.read_csv(f"{QSCORE_PATH}scores.csv", sep=";", low_memory=False)
data_df["image_path"] = f"{IMAGES_PATH}{cfg.YEAR_STUDIED}/" + data_df["image_path"]
data_df["image_id"] = data_df["image_id"].apply(str)
data_df.sort_values(by=[cfg.PERCEPTION_METRIC], ascending=False, inplace=True)
data_df

In [ ]:
data_df = pd.merge(data_df, features_df, how="inner", on=["image_id", "image_path"])
data_df

In [ ]:
%%time
from py.models.datasets import PlacePulse

pp = PlacePulse(data_df)
pp.DataPreparation(delta=cfg.DELTA, emotion=cfg.PERCEPTION_METRIC)
pp.TaskPreparation(task_type=cfg.ML_TASK)
pp.DataSplit()

print(f"Train samples: {len(pp.train_df)}")
print(f"Test samples: {len(pp.test_df)}")

In [ ]:
label_map = dict(zip(pp.data_df["target"], pp.data_df["label"]))
labels = list(label_map.values())
classes = list(label_map.keys())
label_map

In [ ]:
X_train = np.array(pp.train_df["gist"].tolist())
X_test = np.array(pp.test_df["gist"].tolist())
y_train = np.array(pp.train_df["target"].tolist())
y_test = np.array(pp.test_df["target"].tolist())
X_train.shape, X_test.shape

In [ ]:
collections.Counter(y_train), collections.Counter(y_test)

### GridSearch

In [ ]:
from py.models.classification.linear import LinearClassifier

In [ ]:
model_search = LinearClassifier()

#### Train

In [ ]:
%%time
results = model_search.fit_all(X_train, y_train)

In [ ]:
model_search.print_results()

In [ ]:
summary_df = model_search.get_results_summary()
summary_df

#### Inference

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
y_pred = model_search.predict(X_test) # model_name='logistic_regression')

In [ ]:
clf_lr_report = classification_report(y_test, y_pred, output_dict=True)

print(classification_report(y_test, y_pred, target_names=labels))

In [ ]:
confusion_matrix = pd.crosstab(
    [ label_map[k] for k in y_test ],
    [ label_map[k] for k in y_pred ],
    rownames=['Real'],
    colnames=['Prediction']
)
confusion_matrix

#### Saving

In [ ]:
# Save grid search
model_search.save(f'{MODEL_PATH}model_search.pkl')

In [ ]:
# Save just the best trained model
model_search.save_best_model_only(f'{MODEL_PATH}{model_search.get_best_model_name()}_best_model.pkl')

#### Loading

In [ ]:
loaded_search = LinearClassifier.load(f'{MODEL_PATH}model_search.pkl')
y_pred = loaded_search.predict(X_test)
confusion_matrix = pd.crosstab(
    [ label_map[k] for k in y_test ],
    [ label_map[k] for k in y_pred ],
    rownames=['Real'],
    colnames=['Prediction']
)
confusion_matrix

In [ ]:
# Load and use the best model
best_model = LinearClassifier.load_model(f'{MODEL_PATH}{model_search.get_best_model_name()}_best_model.pkl')
y_pred = best_model.predict(X_test)
confusion_matrix = pd.crosstab(
    [ label_map[k] for k in y_test ],
    [ label_map[k] for k in y_pred ],
    rownames=['Real'],
    colnames=['Prediction']
)
confusion_matrix